In [1]:
%load_ext autoreload
%autoreload 2
%aimport - pulp
%aimport - pulp.pulp

# Third-party imports
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from index_travel_accessibility.travel_time_and_centroid import (
    map_predicted_and_existing_hospitals,
)

# Local imports
from ai_planner.planner import (
    Planner,
    CRS
)

from index_travel_accessibility.travel_time_and_centroid import (
    get_hospital_df,
)
from main import (
    Index,
    assemble_indexes,
    centroid,
    current_hospital_demand,
    equity_index,
)


### Create Equity Index

In [2]:
index_df = assemble_indexes()
# Set the weights for the indexes
weight = {
    Index.FORECAST_DEMAND: 0.25,
    Index.ELDERLY_SHARE: 0.25,
    Index.GISD: 0.25,
    Index.HOSPITAL_CAPACITY: 0.25,
    Index.TRAVEL_TIME: 0.25,
    Index.ACCESSIBILITY: 0.25,
}
equity_df = equity_index(index_df, weight)

/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/stats

ARIMA(3,0,0) AIC=-135.65
Current best order (3,0,0)
ARIMA(1,1,0) AIC=429.95
Current best order (1,1,0)
ARIMA(1,1,0) AIC=432.49
Current best order (1,1,0)
ARIMA(1,1,0) AIC=452.99
Current best order (1,1,0)


/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[1:] = col.iloc[1:].map(lambda x: int(x.split('/')[0]))
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for

ARIMA(1,1,0) AIC=439.17
Current best order (1,1,0)
ARIMA(1,1,0) AIC=419.81
Current best order (1,1,0)
ARIMA(1,1,0) AIC=467.72
Current best order (1,1,0)
[GISD] Wrote 6 Saarland districts to /home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_gisd/data/processed/gisd_norm_saarland.csv


/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


********************************************************************************
Estimated time to run the pipeline ---->  2-3 mins
********************************************************************************
✅ Saved Saarland Hospital Dataset
********************************************************************************
✅ Saved Hospital Coordinates Dataset
********************************************************************************
✅ Saved 5 evenly spread sample points per district
********************************************************************************
✅ Found nearest hospitals to all sample points.
********************************************************************************
✅ Generated Travel times from sample points to the nearest hospitals
********************************************************************************
✅ Calculated MEAN, MEDIAN and 95th PERCENTILE of the Travel time for EACH district
************************************************************

### Create Centroids Dataframe

#### Create 1 Centroid per Dataframe

In [8]:

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'index': 'district_code'})


/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[1:] = col.iloc[1:].map(lambda x: int(x.split('/')[0]))
/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_travel_accessibility/travel_time_and_centroid.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use

#### Create 5 Centroids per District

In [5]:
# Load the Excel file
file_path = "./index_travel_accessibility/data/processed/saarland_districts_sample_points.xlsx"
df_sample_points = pd.read_excel(file_path)
SAARLAND_DISTRICT_MAPPING = {
    "Saarlouis": 10044,
    "St. Wendel": 10046,
    "Saarpfalz-Kreis": 10045,
    "Merzig-Wadern": 10042,
    "Neunkirchen": 10043,
    "Regionalverband Saarbrücken": 10041 
}
# Display basic information about the dataframe
df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
df_sample_points = df_sample_points.set_index('district')
curr_hospital_demand = current_hospital_demand()

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
districts_df = df_sample_points.join(districts_df[['demand', 'equity_index']], how='left')
districts_df["centroid"] = districts_df.apply(
    lambda row: Point(row["longitude"], row["latitude"]),
    axis=1
)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'index': 'district_code'})


/tmp/ipykernel_7916/1285653221.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
/home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_demand_forecast/demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

### Executing Planner

In [9]:
#STEP1: Generate Candidates and Initialize Graph
n_samples_per_centroid = 2
radius_km = 7
planner = Planner(districts_gdf=districts_gdf)
planner.generate_candidates(n_samples_per_centroid=n_samples_per_centroid, radius_km=radius_km)
planner.init_graph()
# Save the graph file first to avoid re-downloading the graph
#G = ox.graph_from_place("Saarland, Germany", network_type='drive')
#ox.save_graphml(G, "saarland_graph.graphml")
#Planner.init_graph(graph_path="saarland_graph.graphml")

#### Baseline Model
`demand_weight` param to define the weight of current demand in the objective function 
`equity_weight` param to define the weight of the equity index in the objective function
`travel_weight` param to define the weight of travel time in the objection function
`time_threshold` param to define that only candidate points are assigned to a centroid if its travel time is less or equal to the value.


In [10]:
#STEP2: Initialize Baseline Model
planner.init_baseline_model(
    demand_weight=0.5,
    equity_weight=0.5,
    travel_weight=0.5,
    time_threshold=30 #in minutes
)


#### Add Existing Hospital Constraint
Soft constraint that adds penalty to predicted hospitals if they are located near existing hospital 
`supply_weight` param to define the weight for the supply in the objective function
`distance_threshold` param to define the maximum amount of distance for a candidate to be a neighboor of existing hospitals.

In [17]:
# Get hospital data and convert to GeoDataFrame
hospital_df = get_hospital_df()

# Convert to GeoDataFrame using latitude and longitude columns
hospital_gdf = gpd.GeoDataFrame(
    hospital_df, 
    geometry=gpd.points_from_xy(hospital_df.Lon, hospital_df.Lat),
    crs=CRS  # WGS84 coordinate system
)
planner.init_baseline_model()
planner.add_existing_hospitals_constraints(
    gdf=hospital_gdf,
    supply_weight=0.5,
    distance_threshold=10000, #in meters
    num_neighbors=1
)

/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/pulp/pulp.py:1831: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


#### Add Neighbor Constraint
Hard Constraint such that predicted new hospitals are not located near each other within a certain distance defined by `distrance_threshold`

In [12]:
planner.init_baseline_model()
planner.add_neighbor_constraints(
    distance_threshold=7000 #in meters
)   

#### Add Both Constraints

In [13]:
# Get hospital data and convert to GeoDataFrame
hospital_df = get_hospital_df()

# Convert to GeoDataFrame using latitude and longitude columns
hospital_gdf = gpd.GeoDataFrame(
    hospital_df, 
    geometry=gpd.points_from_xy(hospital_df.Lon, hospital_df.Lat),
    crs=CRS  # WGS84 coordinate system
)
planner.init_baseline_model()
planner.add_existing_hospitals_constraints(
    gdf=hospital_gdf,
    supply_weight=0.5,
    distance_threshold=7000, #in meters
    num_neighbors=1
)
planner.add_neighbor_constraints(
    distance_threshold=7000 #in meters
)   

/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/pulp/pulp.py:1831: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


#### Set the number of prediction

In [18]:
#STEP3: Set Number of Predictions
planner.set_num_predictions(k=10)

#### Predict and Plot

In [19]:
selected, assigns = planner.predict()
selected_candidates = planner.candidates_gdf.iloc[selected]
# Add Lat and Lon columns using assign
# selected_candidates have Lat and Lon of the new predicted hospitals
selected_candidates = selected_candidates.assign(
    Lat=selected_candidates.geometry.y,
    Lon=selected_candidates.geometry.x
)
map_predicted_and_existing_hospitals(
    'results/test.html', 
    selected_candidates
)

✅ Map saved with existing and predicted hospitals!
